# JuSpyce API test: null maps generation & permutation test

In [2]:
import sys
import os
from glob import glob
import pathlib
import numpy as np
import pandas as pd
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.api import JuSpyce
from juspyce.stats import *
from juspyce.utils import *

/Users/llotter/projects/juspyce


## Load JuSpyce data from test_juspyce.fit.ipynb

In [3]:
juspyce_vol = JuSpyce.from_pickle(os.path.join(wd, "testing", "test_juspyce_vol_predicted.pkl.gz"))

INFO:juspyce.api:Loaded complete object from /Users/llotter/projects/juspyce/testing/test_juspyce_vol_predicted.pkl.gz.


## Permutation based on null maps

Uses the method from brainsmash based on variograms to correct for spatial autocorrelation. Distance maps are based on geodesic distances for surface parcellations and euclidean distances for volumetric parcellations.

One has to choose a method for which empirical p-values will be calculated. If the `JuSpyce.permute_maps()` method is then applied again for computation of another metric, the already existing null maps will be used (this behavior can be turned off).

In [5]:
# everything below that is set to None will be set to customs or taken from the .fit() method
_ = juspyce_vol.permute_maps(
    method="dominance", # which method
    permute="X", # null maps for which dataset? Can be 'X' or 'Y'
    null_method="variogram", # "variogram" -> brainsmash, "random" -> np.random
    null_maps=None, # directly provide null maps. Must be dict with keys corresponding to the maps
                    # for which null data is generated (e.g., "mGluR5" or so)
    use_null_maps=True, # re-use null maps if already computed for another prediction method
    dist_mat=None, # custom distance matrix -> must be ndarray with shape (n_parcels, n_parcels) 
                   # or tuple of those ndarrays if parcellation is tuple of giftis
    n_perm=1000, # number if permutations (= number of null maps)
    parcellation=None, parc_space=None, parc_hemi=None, # parcellation data, usually set at .fit()
    centroids=False, # compute distance matrices between parcel centroids? -> faster
    r_to_z=None, adjust_r2=None, mlr_individual=None, # settings for prediction (see above)
    p_tail=None, # which-sided p-value? Must be dict with e.g. 
                 # {"spearman":"two"} or 
                 # {"dominance_total":"upper", "dominance_relative":"upper", 
                 #  "dominance_full_r2":"upper", "dominance_individual":"upper"}, etc.
    n_proc=None, # number of processes
    n_proc_predict=1, # number of processes for prediction method -> multiplies itself with n_proc!
    seed=None, # seed for reproducability
    store=True)

Null predictions (dominance, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = {'dominance_total': 'upper', 'dominance_individual': 'upper', 'dominance_relative': 'upper', 'dominance_full_r2': 'upper'}).


### Print result

All results are stored in a dict as `JuSpyce.p_predictions["prediction_name"]` which corresponds to `JuSpyce.predictions["prediction_name"]`

In [6]:
for dom in ["dominance_full_r2", "dominance_relative", "dominance_total", "dominance_individual"]:
    print(dom)
    display(juspyce_vol.p_predictions[dom])

dominance_full_r2


,dominance_full_r2
control,0.003
touch,0.102
interoception,0.048
learning,0.001
attention,0.732
language,0.495
interaction,0.097
inhibition,0.068
somatosensory,0.104
decision,0.001


dominance_relative


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.166,0.774,0.426,0.462,0.230,0.542,0.374
touch,0.223,0.602,0.109,0.473,0.523,0.673,0.324
interoception,0.790,0.206,0.095,0.232,0.710,0.727,0.449
learning,0.423,0.529,0.189,0.618,0.494,0.879,0.094
attention,0.842,0.490,0.643,0.006,0.708,0.829,0.482
language,0.035,0.830,0.292,0.654,0.452,0.405,0.812
interaction,0.586,0.660,0.351,0.004,0.737,0.893,0.790
inhibition,0.150,0.668,0.377,0.512,0.301,0.403,0.430
somatosensory,0.261,0.563,0.075,0.478,0.552,0.726,0.333
decision,0.694,0.787,0.663,0.290,0.476,0.114,0.113


dominance_total


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.030,0.603,0.234,0.262,0.070,0.369,0.152
touch,0.121,0.456,0.025,0.320,0.390,0.564,0.173
interoception,0.693,0.078,0.028,0.117,0.600,0.618,0.306
learning,0.045,0.095,0.003,0.211,0.064,0.541,0.001
attention,0.852,0.516,0.674,0.009,0.726,0.850,0.517
language,0.061,0.810,0.287,0.651,0.426,0.391,0.805
interaction,0.448,0.535,0.230,0.001,0.632,0.831,0.704
inhibition,0.067,0.532,0.231,0.355,0.166,0.270,0.269
somatosensory,0.140,0.419,0.018,0.334,0.423,0.615,0.178
decision,0.224,0.344,0.231,0.009,0.049,0.001,0.001


dominance_individual


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.036,0.727,0.167,0.594,0.055,0.817,0.943
touch,0.349,0.476,0.416,0.510,0.690,0.460,0.473
interoception,0.660,0.035,0.040,0.020,0.935,0.641,0.139
learning,0.001,0.001,0.001,0.028,0.001,0.741,0.001
attention,0.685,0.395,0.802,0.014,0.733,0.816,0.370
language,0.143,0.733,0.269,0.810,0.681,0.348,0.809
interaction,0.551,0.981,0.147,0.006,0.918,0.685,0.750
inhibition,0.104,0.620,0.615,0.740,0.183,0.574,0.997
somatosensory,0.420,0.425,0.286,0.498,0.803,0.528,0.440
decision,0.018,0.219,0.344,0.001,0.001,0.001,0.001


## Correct p-values

p values can be corrected across dataframes or rows/columns of dataframes using `JuSpyce.correct_p()`. The method will, if not provided differently, loop over all p-value dataframes and apply multiple comparison correction methods from `statsmodels.stats.multitest.multipletests`.

Results will be stored in the `JuSpyce.p_predictions` dict as `JuSpyce.p_predictions`["prediction_name-correction_method"], e.g., if method is `spearman` and correction is `fdr_bh:` juspyce_vol.p_predictions["spearman-fdr_bh"]`

In [8]:
juspyce_vol.correct_p(
    analysis="predictions", # one of "predictions" or "group_comparisons" -> here: predictions
    method="all", # if all, iterate over all dataframes (but calculate values for each individual dataframe)
    mc_alpha=0.05, # alpha treshold, should have no effect
    mc_method="fdr_bh", # correction method passed to statsmodels
    mc_dimension="array") # 'array', 'row' or 'column'
display(juspyce_vol.p_predictions["dominance_total-fdr_bh"])

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.189677,0.733185,0.504000,0.540000,0.298261,0.602700,0.419222
touch,0.370562,0.662044,0.175000,0.575413,0.623057,0.721580,0.446158
interoception,0.771750,0.316615,0.189241,0.364000,0.733185,0.734109,0.565811
learning,0.252000,0.344815,0.036750,0.504000,0.291721,0.706907,0.017818
attention,0.869750,0.694055,0.761471,0.088200,0.790391,0.869750,0.694055
language,0.291721,0.848984,0.556951,0.751718,0.640366,0.623057,0.848280
interaction,0.660211,0.703758,0.504000,0.017818,0.741749,0.866362,0.779571
inhibition,0.296178,0.703758,0.504000,0.589661,0.433813,0.540000,0.540000
somatosensory,0.397681,0.640366,0.141120,0.584500,0.640366,0.734109,0.453091
decision,0.504000,0.584650,0.504000,0.088200,0.257895,0.017818,0.017818
